## Setup

### Environment Configuration

Before running this notebook, ensure you have the following:
- TensorFlow 2.x installed
- The pose NPZ files generated (both static and temporal versions)
- Config files for experiment 7 in the `config/` directory

### NPZ Files Required
- `pose_data_front_temporal.npz` - Front view with 50 temporal frames per sample
- `pose_data_side_temporal.npz` - Side view with 50 temporal frames per sample
- `pose_data_front_static.npz` - Front view with 5 statistics per angle
- `pose_data_side_static.npz` - Side view with 5 statistics per angle

### Imports and Project Path Configuration

In [1]:
# Core imports
import sys
import os
from pathlib import Path

# Data and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set project root
PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"Working directory: {Path.cwd()}")

Project root: /mnt/d/Graduation_Project/ai-virtual-coach
Working directory: /mnt/d/Graduation_Project/ai-virtual-coach/notebooks/exer_recog


In [2]:
# Reload modules for development
%load_ext autoreload
%autoreload 2

In [4]:
# Import project modules
from src.data import load_pose_data
from src.data.data_loader import load_pose_temporal_data
from src.scripts.experiment_7 import (
    train_experiment_7_temporal,
    train_experiment_7_temporal_multi_run,
    train_experiment_7_static,
    train_experiment_7_static_multi_run
)
from src.utils.visualization import (
    plot_training_history,
    plot_confusion_matrix_from_metrics
)

print("All modules imported successfully!")

All modules imported successfully!


### Define Data Paths and Config Paths

In [5]:
# NPZ file paths
POSE_DATA_DIR = PROJECT_ROOT / 'datasets' / 'Mediapipe pose estimates'

# Temporal NPZ files (native LSTM input)
front_temporal_npz = POSE_DATA_DIR / 'pose_data_front_temporal.npz'
side_temporal_npz = POSE_DATA_DIR / 'pose_data_side_temporal.npz'

# Static NPZ files (reshaped for LSTM)
front_static_npz = POSE_DATA_DIR / 'pose_data_front_static.npz'
side_static_npz = POSE_DATA_DIR / 'pose_data_side_static.npz'

# Config paths
config_path_temporal_front = PROJECT_ROOT / 'config' / 'experiment_7_temporal_front.yaml'
config_path_temporal_side = PROJECT_ROOT / 'config' / 'experiment_7_temporal_side.yaml'
config_path_static_front = PROJECT_ROOT / 'config' / 'experiment_7_static_front.yaml'
config_path_static_side = PROJECT_ROOT / 'config' / 'experiment_7_static_side.yaml'

print("Data paths configured:")
print(f"  Temporal Front NPZ: {front_temporal_npz}")
print(f"  Temporal Side NPZ:  {side_temporal_npz}")
print(f"  Static Front NPZ:   {front_static_npz}")
print(f"  Static Side NPZ:    {side_static_npz}")

Data paths configured:
  Temporal Front NPZ: /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_front_temporal.npz
  Temporal Side NPZ:  /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_temporal.npz
  Static Front NPZ:   /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_front_static.npz
  Static Side NPZ:    /mnt/d/Graduation_Project/ai-virtual-coach/datasets/Mediapipe pose estimates/pose_data_side_static.npz


### Quick Data Inspection

In [8]:
# Load and inspect temporal data
front_temporal_dataset, front_temporal_summary = load_pose_temporal_data(str(front_temporal_npz))

print("Front Temporal Dataset Summary:")
print(f"  Total samples: {front_temporal_summary['count']}")
print(f"  Temporal shape: {front_temporal_summary['temporal_shape']}")
print(f"  Angle names: {front_temporal_summary.get('angle_names', 'Not available')}")
print(f"  Unique classes: {front_temporal_summary['unique_classes']}")

INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)


Front Temporal Dataset Summary:
  Total samples: 1574
  Temporal shape: (50, 9)
  Angle names: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'torso_lean']
  Unique classes: 15


In [9]:
# Load and inspect static data
front_static_dataset, front_static_summary = load_pose_data(str(front_static_npz))

print("Front Static Dataset Summary:")
print(f"  Total samples: {front_static_summary['count']}")
print(f"  Unique subjects: {front_static_summary['unique_subjects']}")
print(f"  Angle names: {front_static_summary.get('angle_names', 'Not available')}")
print(f"  Unique classes: {front_static_summary['unique_classes']}")

INFO - [load_pose_data] Loaded 1574 samples (49 subjects, 15 classes) from front view


Front Static Dataset Summary:
  Total samples: 1574
  Unique subjects: 49
  Angle names: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'torso_lean']
  Unique classes: 15


---

# Part 1: Temporal LSTM Training

LSTM with native sequential input from temporal pose features.

**Input shape:** `(50 timesteps, N angles)` where N depends on config's `selected_angles`

## 1.1 Front View - Temporal LSTM

In [10]:
# Run multi-run training for front temporal LSTM
os.chdir(PROJECT_ROOT)  # Ensure we're in project root for relative paths in config

multi_run_results_temporal_front, aggregated_stats_temporal_front = train_experiment_7_temporal_multi_run(
    npz_path=str(front_temporal_npz),
    config_path=str(config_path_temporal_front)
)

INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Multi-run parent folder: /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001
INFO - 
INFO - Starting run 1/30 (seed=43)
INFO - ================================================================================
INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Global random seed set to: 43
INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)
INFO - Filtered temporal features for LSTM: 8 angles selected (left_elbow, right_elbow, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee) -> shape (50, 8)
INFO - Using selected angles: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', 'left_knee', 'right_knee']
I

Epoch 1/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 242ms/step - accuracy: 0.2020 - loss: 2.5744 - val_accuracy: 0.4206 - val_loss: 2.3447
Epoch 2/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 223ms/step - accuracy: 0.3362 - loss: 2.2907 - val_accuracy: 0.5159 - val_loss: 2.0045
Epoch 3/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 212ms/step - accuracy: 0.4116 - loss: 1.9848 - val_accuracy: 0.5952 - val_loss: 1.6578
Epoch 4/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 200ms/step - accuracy: 0.4913 - loss: 1.7155 - val_accuracy: 0.6349 - val_loss: 1.3862
Epoch 5/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - accuracy: 0.5371 - loss: 1.5229 - val_accuracy: 0.7222 - val_loss: 1.1890
Epoch 6/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 208ms/step - accuracy: 0.5546 - loss: 1.3578 - val_accuracy: 0.7103 - val_loss: 1.1299
Epoch 7/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 203ms/step - accuracy: 0.5862 - loss: 1.2294 - val_accuracy: 0.7262 - val_loss: 1.0887
Epoch 8/80
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - accuracy: 0.5983 - loss: 1.1833 - val_accuracy: 0.

INFO - Saved metrics to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/run_001/metrics.json
INFO - Experiment 7 (LSTM temporal) complete. Test acc=0.6675, macro F1=0.6579
INFO - Run 1 complete: acc=0.6675, macro_f1=0.6579
INFO - 
INFO - Starting run 2/30 (seed=44)
INFO - ================================================================================
INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Global random seed set to: 44
INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)
INFO - Filtered temporal features for LSTM: 8 angles selected (left_elbow, right_elbow, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee) -> shape (50, 8)
INFO - Using selected angles: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', '

Epoch 1/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 214ms/step - accuracy: 0.0955 - loss: 2.6451 - val_accuracy: 0.2050 - val_loss: 2.5323
Epoch 2/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - accuracy: 0.2807 - loss: 2.3909 - val_accuracy: 0.3000 - val_loss: 2.2923
Epoch 3/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.3844 - loss: 2.1250 - val_accuracy: 0.3400 - val_loss: 2.0387
Epoch 4/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 195ms/step - accuracy: 0.4116 - loss: 1.8639 - val_accuracy: 0.3100 - val_loss: 1.8919
Epoch 5/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - accuracy: 0.4917 - loss: 1.6363 - val_accuracy: 0.3950 - val_loss: 1.7538
Epoch 6/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 182ms/step - accuracy: 0.5566 - loss: 1.4593 - val_accuracy: 0.4250 - val_loss: 1.7112
Epoch 7/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 276ms/step - accuracy: 0.5625 - loss: 1.3723 - val_accuracy: 0.4950 - val_loss: 1.5962
Epoch 8/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.6014 - loss: 1.2494 - val_accuracy: 0.

INFO - Saved metrics to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/run_002/metrics.json
INFO - Experiment 7 (LSTM temporal) complete. Test acc=0.6521, macro F1=0.6495
INFO - Run 2 complete: acc=0.6521, macro_f1=0.6495
INFO - 
INFO - Starting run 3/30 (seed=45)
INFO - ================================================================================
INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Global random seed set to: 45
INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)
INFO - Filtered temporal features for LSTM: 8 angles selected (left_elbow, right_elbow, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee) -> shape (50, 8)
INFO - Using selected angles: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip', '

...

INFO - Saved metrics to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/run_028/metrics.json
INFO - Experiment 7 (LSTM temporal) complete. Test acc=0.5899, macro F1=0.5594
INFO - Run 28 complete: acc=0.5899, macro_f1=0.5594
INFO - 
INFO - Starting run 29/30 (seed=71)
INFO - ================================================================================
INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Global random seed set to: 71
INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)
INFO - Filtered temporal features for LSTM: 8 angles selected (left_elbow, right_elbow, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee) -> shape (50, 8)
INFO - Using selected angles: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',

Epoch 1/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 171ms/step - accuracy: 0.1819 - loss: 2.5727 - val_accuracy: 0.2717 - val_loss: 2.4094
Epoch 2/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 157ms/step - accuracy: 0.3155 - loss: 2.2778 - val_accuracy: 0.3121 - val_loss: 2.1166
Epoch 3/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.3823 - loss: 2.0212 - val_accuracy: 0.4566 - val_loss: 1.8226
Epoch 4/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step - accuracy: 0.4789 - loss: 1.7664 - val_accuracy: 0.5491 - val_loss: 1.5753
Epoch 5/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 241ms/step - accuracy: 0.5139 - loss: 1.5383 - val_accuracy: 0.5896 - val_loss: 1.3431
Epoch 6/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - accuracy: 0.5406 - loss: 1.3925 - val_accuracy: 0.6763 - val_loss: 1.1676
Epoch 7/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.5725 - loss: 1.2888 - val_accuracy: 0.6936 - val_loss: 1.0471
Epoch 8/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.5951 - loss: 1.2192 - val_accuracy: 0.

INFO - Saved metrics to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/run_029/metrics.json
INFO - Experiment 7 (LSTM temporal) complete. Test acc=0.6612, macro F1=0.6295
INFO - Run 29 complete: acc=0.6612, macro_f1=0.6295
INFO - 
INFO - Starting run 30/30 (seed=72)
INFO - ================================================================================
INFO - Configuration loaded from /mnt/d/Graduation_Project/ai-virtual-coach/config/experiment_7_temporal_front.yaml
INFO - Global random seed set to: 72
INFO - [load_pose_temporal_data] Loaded 1574 samples (49 subjects, 15 classes) from front view, temporal shape: (50, 9)
INFO - Filtered temporal features for LSTM: 8 angles selected (left_elbow, right_elbow, left_shoulder, right_shoulder, left_hip, right_hip, left_knee, right_knee) -> shape (50, 8)
INFO - Using selected angles: ['left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',

Epoch 1/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 297ms/step - accuracy: 0.1624 - loss: 2.5836 - val_accuracy: 0.2045 - val_loss: 2.4106
Epoch 2/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.2431 - loss: 2.3291 - val_accuracy: 0.2689 - val_loss: 2.1474
Epoch 3/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.3065 - loss: 2.0969 - val_accuracy: 0.2386 - val_loss: 1.9583
Epoch 4/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - accuracy: 0.3468 - loss: 1.9237 - val_accuracy: 0.3939 - val_loss: 1.7679
Epoch 5/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.4055 - loss: 1.7756 - val_accuracy: 0.4886 - val_loss: 1.6110
Epoch 6/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - accuracy: 0.4816 - loss: 1.5925 - val_accuracy: 0.5985 - val_loss: 1.4034
Epoch 7/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - accuracy: 0.4931 - loss: 1.4938 - val_accuracy: 0.6250 - val_loss: 1.3003
Epoch 8/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 157ms/step - accuracy: 0.5346 - loss: 1.4197 - val_accuracy: 0.

INFO - Saved metrics to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/run_030/metrics.json
INFO - Experiment 7 (LSTM temporal) complete. Test acc=0.7398, macro F1=0.7257
INFO - Run 30 complete: acc=0.7398, macro_f1=0.7257
INFO - 
INFO - Computing aggregated statistics across 30 runs
INFO - ================================================================================
INFO - Saved aggregated stats to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/aggregated_stats.json
INFO - Saved summary to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/aggregated_summary.txt
INFO - Saved all run results to /mnt/d/Graduation_Project/ai-virtual-coach/experiments/exer_recog/results/exp_07_pose_lstm_temporal/front/multi_run_001/all_runs.json
INFO - 
INFO - MULTI-RUN EXPERIMENT 7 (LSTM TE

### Temporal Front Results Analysis

In [ ]:
# Display aggregated statistics
print("=" * 60)
print("TEMPORAL FRONT LSTM - Multi-Run Results (30 runs)")
print("=" * 60)
print(f"Test Accuracy: {aggregated_stats_temporal_front['test_accuracy']['mean']:.4f} ± {aggregated_stats_temporal_front['test_accuracy']['std']:.4f}")
print(f"Test Macro F1: {aggregated_stats_temporal_front['test_macro_f1']['mean']:.4f} ± {aggregated_stats_temporal_front['test_macro_f1']['std']:.4f}")
print(f"Best Accuracy: {aggregated_stats_temporal_front['test_accuracy']['max']:.4f}")
print(f"Best Macro F1: {aggregated_stats_temporal_front['test_macro_f1']['max']:.4f}")

In [ ]:
# Find best run by macro F1
best_run_temporal_front = max(multi_run_results_temporal_front, key=lambda x: x['test_metrics']['macro_f1'])
plot_training_history(best_run_temporal_front['history'], title="Temporal Front LSTM - Best Run Training History")

In [ ]:
# Confusion matrix from best run
conf_matrix_temporal_front = np.array(best_run_temporal_front['test_metrics']['confusion_matrix'])
idx_to_label_temporal_front = best_run_temporal_front['int_to_label']
label_names_temporal_front = [idx_to_label_temporal_front[i] for i in range(len(idx_to_label_temporal_front))]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(conf_matrix_temporal_front, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_names_temporal_front, yticklabels=label_names_temporal_front, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Temporal Front LSTM - Confusion Matrix (Best Run)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Per-class F1 scores
per_class_f1_temporal_front = pd.Series(aggregated_stats_temporal_front['per_class_f1']).apply(
    lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}"
)
per_class_f1_temporal_front.index = [idx_to_label_temporal_front[i] for i in per_class_f1_temporal_front.index]

summary_df_temporal_front = pd.DataFrame({
    'Mean F1 ± Std': per_class_f1_temporal_front
})
print("Per-Class F1 Scores (Temporal Front LSTM):")
print(summary_df_temporal_front.to_string())

## 1.2 Side View - Temporal LSTM

In [ ]:
# Run multi-run training for side temporal LSTM
os.chdir(PROJECT_ROOT)

multi_run_results_temporal_side, aggregated_stats_temporal_side = train_experiment_7_temporal_multi_run(
    npz_path=str(side_temporal_npz),
    config_path=str(config_path_temporal_side)
)

### Temporal Side Results Analysis

In [ ]:
# Display aggregated statistics
print("=" * 60)
print("TEMPORAL SIDE LSTM - Multi-Run Results (30 runs)")
print("=" * 60)
print(f"Test Accuracy: {aggregated_stats_temporal_side['test_accuracy']['mean']:.4f} ± {aggregated_stats_temporal_side['test_accuracy']['std']:.4f}")
print(f"Test Macro F1: {aggregated_stats_temporal_side['test_macro_f1']['mean']:.4f} ± {aggregated_stats_temporal_side['test_macro_f1']['std']:.4f}")
print(f"Best Accuracy: {aggregated_stats_temporal_side['test_accuracy']['max']:.4f}")
print(f"Best Macro F1: {aggregated_stats_temporal_side['test_macro_f1']['max']:.4f}")

In [ ]:
# Find best run
best_run_temporal_side = max(multi_run_results_temporal_side, key=lambda x: x['test_metrics']['macro_f1'])
plot_training_history(best_run_temporal_side['history'], title="Temporal Side LSTM - Best Run Training History")

In [ ]:
# Confusion matrix from best run
conf_matrix_temporal_side = np.array(best_run_temporal_side['test_metrics']['confusion_matrix'])
idx_to_label_temporal_side = best_run_temporal_side['int_to_label']
label_names_temporal_side = [idx_to_label_temporal_side[i] for i in range(len(idx_to_label_temporal_side))]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(conf_matrix_temporal_side, annot=True, fmt='d', cmap='Greens',
            xticklabels=label_names_temporal_side, yticklabels=label_names_temporal_side, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Temporal Side LSTM - Confusion Matrix (Best Run)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Per-class F1 scores
per_class_f1_temporal_side = pd.Series(aggregated_stats_temporal_side['per_class_f1']).apply(
    lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}"
)
per_class_f1_temporal_side.index = [idx_to_label_temporal_side[i] for i in per_class_f1_temporal_side.index]

summary_df_temporal_side = pd.DataFrame({
    'Mean F1 ± Std': per_class_f1_temporal_side
})
print("Per-Class F1 Scores (Temporal Side LSTM):")
print(summary_df_temporal_side.to_string())

### Temporal Front vs Side Comparison

In [ ]:
# Build comparison DataFrame for temporal models
comparison_df_temporal = pd.DataFrame({
    'Model': ['Temporal LSTM Front', 'Temporal LSTM Side'],
    'Test Accuracy (mean)': [
        aggregated_stats_temporal_front['test_accuracy']['mean'],
        aggregated_stats_temporal_side['test_accuracy']['mean']
    ],
    'Test Accuracy (std)': [
        aggregated_stats_temporal_front['test_accuracy']['std'],
        aggregated_stats_temporal_side['test_accuracy']['std']
    ],
    'Test Macro F1 (mean)': [
        aggregated_stats_temporal_front['test_macro_f1']['mean'],
        aggregated_stats_temporal_side['test_macro_f1']['mean']
    ],
    'Test Macro F1 (std)': [
        aggregated_stats_temporal_front['test_macro_f1']['std'],
        aggregated_stats_temporal_side['test_macro_f1']['std']
    ],
})

print("Temporal LSTM Models Comparison:")
print(comparison_df_temporal.to_string(index=False))

In [ ]:
# Visualize temporal models comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = ['Temporal Front', 'Temporal Side']
acc_means = [aggregated_stats_temporal_front['test_accuracy']['mean'],
             aggregated_stats_temporal_side['test_accuracy']['mean']]
acc_stds = [aggregated_stats_temporal_front['test_accuracy']['std'],
            aggregated_stats_temporal_side['test_accuracy']['std']]

f1_means = [aggregated_stats_temporal_front['test_macro_f1']['mean'],
            aggregated_stats_temporal_side['test_macro_f1']['mean']]
f1_stds = [aggregated_stats_temporal_front['test_macro_f1']['std'],
           aggregated_stats_temporal_side['test_macro_f1']['std']]

colors = ['steelblue', 'seagreen']

axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors, alpha=0.8)
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Temporal LSTM - Accuracy Comparison')
axes[0].set_ylim(0, 1)

axes[1].bar(models, f1_means, yerr=f1_stds, capsize=5, color=colors, alpha=0.8)
axes[1].set_ylabel('Test Macro F1')
axes[1].set_title('Temporal LSTM - Macro F1 Comparison')
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

---

# Part 2: Static LSTM Training

LSTM with reshaped static features.

**Input shape:** `(N angles, 5 statistics)` where angles are treated as timesteps.

## 2.1 Front View - Static LSTM

In [ ]:
# Run multi-run training for front static LSTM
os.chdir(PROJECT_ROOT)

multi_run_results_static_front, aggregated_stats_static_front = train_experiment_7_static_multi_run(
    npz_path=str(front_static_npz),
    config_path=str(config_path_static_front)
)

### Static Front Results Analysis

In [ ]:
# Display aggregated statistics
print("=" * 60)
print("STATIC FRONT LSTM - Multi-Run Results (30 runs)")
print("=" * 60)
print(f"Test Accuracy: {aggregated_stats_static_front['test_accuracy']['mean']:.4f} ± {aggregated_stats_static_front['test_accuracy']['std']:.4f}")
print(f"Test Macro F1: {aggregated_stats_static_front['test_macro_f1']['mean']:.4f} ± {aggregated_stats_static_front['test_macro_f1']['std']:.4f}")
print(f"Best Accuracy: {aggregated_stats_static_front['test_accuracy']['max']:.4f}")
print(f"Best Macro F1: {aggregated_stats_static_front['test_macro_f1']['max']:.4f}")

In [ ]:
# Find best run
best_run_static_front = max(multi_run_results_static_front, key=lambda x: x['test_metrics']['macro_f1'])
plot_training_history(best_run_static_front['history'], title="Static Front LSTM - Best Run Training History")

In [ ]:
# Confusion matrix from best run
conf_matrix_static_front = np.array(best_run_static_front['test_metrics']['confusion_matrix'])
idx_to_label_static_front = best_run_static_front['int_to_label']
label_names_static_front = [idx_to_label_static_front[i] for i in range(len(idx_to_label_static_front))]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(conf_matrix_static_front, annot=True, fmt='d', cmap='Oranges',
            xticklabels=label_names_static_front, yticklabels=label_names_static_front, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Static Front LSTM - Confusion Matrix (Best Run)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Per-class F1 scores
per_class_f1_static_front = pd.Series(aggregated_stats_static_front['per_class_f1']).apply(
    lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}"
)
per_class_f1_static_front.index = [idx_to_label_static_front[i] for i in per_class_f1_static_front.index]

summary_df_static_front = pd.DataFrame({
    'Mean F1 ± Std': per_class_f1_static_front
})
print("Per-Class F1 Scores (Static Front LSTM):")
print(summary_df_static_front.to_string())

## 2.2 Side View - Static LSTM

In [ ]:
# Run multi-run training for side static LSTM
os.chdir(PROJECT_ROOT)

multi_run_results_static_side, aggregated_stats_static_side = train_experiment_7_static_multi_run(
    npz_path=str(side_static_npz),
    config_path=str(config_path_static_side)
)

### Static Side Results Analysis

In [ ]:
# Display aggregated statistics
print("=" * 60)
print("STATIC SIDE LSTM - Multi-Run Results (30 runs)")
print("=" * 60)
print(f"Test Accuracy: {aggregated_stats_static_side['test_accuracy']['mean']:.4f} ± {aggregated_stats_static_side['test_accuracy']['std']:.4f}")
print(f"Test Macro F1: {aggregated_stats_static_side['test_macro_f1']['mean']:.4f} ± {aggregated_stats_static_side['test_macro_f1']['std']:.4f}")
print(f"Best Accuracy: {aggregated_stats_static_side['test_accuracy']['max']:.4f}")
print(f"Best Macro F1: {aggregated_stats_static_side['test_macro_f1']['max']:.4f}")

In [ ]:
# Find best run
best_run_static_side = max(multi_run_results_static_side, key=lambda x: x['test_metrics']['macro_f1'])
plot_training_history(best_run_static_side['history'], title="Static Side LSTM - Best Run Training History")

In [ ]:
# Confusion matrix from best run
conf_matrix_static_side = np.array(best_run_static_side['test_metrics']['confusion_matrix'])
idx_to_label_static_side = best_run_static_side['int_to_label']
label_names_static_side = [idx_to_label_static_side[i] for i in range(len(idx_to_label_static_side))]

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(conf_matrix_static_side, annot=True, fmt='d', cmap='Purples',
            xticklabels=label_names_static_side, yticklabels=label_names_static_side, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Static Side LSTM - Confusion Matrix (Best Run)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Per-class F1 scores
per_class_f1_static_side = pd.Series(aggregated_stats_static_side['per_class_f1']).apply(
    lambda x: f"{x['mean']:.4f} ± {x['std']:.4f}"
)
per_class_f1_static_side.index = [idx_to_label_static_side[i] for i in per_class_f1_static_side.index]

summary_df_static_side = pd.DataFrame({
    'Mean F1 ± Std': per_class_f1_static_side
})
print("Per-Class F1 Scores (Static Side LSTM):")
print(summary_df_static_side.to_string())

### Static Front vs Side Comparison

In [ ]:
# Build comparison DataFrame for static models
comparison_df_static = pd.DataFrame({
    'Model': ['Static LSTM Front', 'Static LSTM Side'],
    'Test Accuracy (mean)': [
        aggregated_stats_static_front['test_accuracy']['mean'],
        aggregated_stats_static_side['test_accuracy']['mean']
    ],
    'Test Accuracy (std)': [
        aggregated_stats_static_front['test_accuracy']['std'],
        aggregated_stats_static_side['test_accuracy']['std']
    ],
    'Test Macro F1 (mean)': [
        aggregated_stats_static_front['test_macro_f1']['mean'],
        aggregated_stats_static_side['test_macro_f1']['mean']
    ],
    'Test Macro F1 (std)': [
        aggregated_stats_static_front['test_macro_f1']['std'],
        aggregated_stats_static_side['test_macro_f1']['std']
    ],
})

print("Static LSTM Models Comparison:")
print(comparison_df_static.to_string(index=False))

In [ ]:
# Visualize static models comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = ['Static Front', 'Static Side']
acc_means = [aggregated_stats_static_front['test_accuracy']['mean'],
             aggregated_stats_static_side['test_accuracy']['mean']]
acc_stds = [aggregated_stats_static_front['test_accuracy']['std'],
            aggregated_stats_static_side['test_accuracy']['std']]

f1_means = [aggregated_stats_static_front['test_macro_f1']['mean'],
            aggregated_stats_static_side['test_macro_f1']['mean']]
f1_stds = [aggregated_stats_static_front['test_macro_f1']['std'],
           aggregated_stats_static_side['test_macro_f1']['std']]

colors = ['darkorange', 'purple']

axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors, alpha=0.8)
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Static LSTM - Accuracy Comparison')
axes[0].set_ylim(0, 1)

axes[1].bar(models, f1_means, yerr=f1_stds, capsize=5, color=colors, alpha=0.8)
axes[1].set_ylabel('Test Macro F1')
axes[1].set_title('Static LSTM - Macro F1 Comparison')
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

---

# Part 3: Comprehensive Comparison

Compare all 4 LSTM models (Temporal Front, Temporal Side, Static Front, Static Side)

## All Models Summary Table

In [ ]:
# Comprehensive comparison table
all_models_stats = {
    'Temporal Front LSTM': aggregated_stats_temporal_front,
    'Temporal Side LSTM': aggregated_stats_temporal_side,
    'Static Front LSTM': aggregated_stats_static_front,
    'Static Side LSTM': aggregated_stats_static_side,
}

comparison_data = []
for model_name, stats in all_models_stats.items():
    comparison_data.append({
        'Model': model_name,
        'Test Accuracy (mean ± std)': f"{stats['test_accuracy']['mean']:.4f} ± {stats['test_accuracy']['std']:.4f}",
        'Test Macro F1 (mean ± std)': f"{stats['test_macro_f1']['mean']:.4f} ± {stats['test_macro_f1']['std']:.4f}",
        'Best Accuracy': f"{stats['test_accuracy']['max']:.4f}",
        'Best Macro F1': f"{stats['test_macro_f1']['max']:.4f}",
    })

comparison_df_all = pd.DataFrame(comparison_data)
print("=" * 100)
print("EXPERIMENT 7 - ALL LSTM MODELS COMPARISON")
print("=" * 100)
print(comparison_df_all.to_string(index=False))

In [ ]:
# Identify best model
best_model_name = max(all_models_stats.keys(), 
                      key=lambda x: all_models_stats[x]['test_macro_f1']['mean'])
best_f1 = all_models_stats[best_model_name]['test_macro_f1']['mean']
best_f1_std = all_models_stats[best_model_name]['test_macro_f1']['std']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Macro F1: {best_f1:.4f} ± {best_f1_std:.4f}")

## All Models Bar Chart Comparison

In [ ]:
# Comprehensive bar chart comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

model_names = ['Temporal\nFront', 'Temporal\nSide', 'Static\nFront', 'Static\nSide']
colors = ['steelblue', 'seagreen', 'darkorange', 'purple']

# Accuracy comparison
acc_means = [
    aggregated_stats_temporal_front['test_accuracy']['mean'],
    aggregated_stats_temporal_side['test_accuracy']['mean'],
    aggregated_stats_static_front['test_accuracy']['mean'],
    aggregated_stats_static_side['test_accuracy']['mean']
]
acc_stds = [
    aggregated_stats_temporal_front['test_accuracy']['std'],
    aggregated_stats_temporal_side['test_accuracy']['std'],
    aggregated_stats_static_front['test_accuracy']['std'],
    aggregated_stats_static_side['test_accuracy']['std']
]

bars1 = axes[0].bar(model_names, acc_means, yerr=acc_stds, capsize=5, color=colors, alpha=0.8)
axes[0].set_ylabel('Test Accuracy', fontsize=12)
axes[0].set_title('All LSTM Models - Accuracy Comparison', fontsize=14)
axes[0].set_ylim(0, 1)
axes[0].axhline(y=max(acc_means), color='red', linestyle='--', alpha=0.5, label='Best')

# Add value labels
for bar, mean, std in zip(bars1, acc_means, acc_stds):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=10)

# Macro F1 comparison
f1_means = [
    aggregated_stats_temporal_front['test_macro_f1']['mean'],
    aggregated_stats_temporal_side['test_macro_f1']['mean'],
    aggregated_stats_static_front['test_macro_f1']['mean'],
    aggregated_stats_static_side['test_macro_f1']['mean']
]
f1_stds = [
    aggregated_stats_temporal_front['test_macro_f1']['std'],
    aggregated_stats_temporal_side['test_macro_f1']['std'],
    aggregated_stats_static_front['test_macro_f1']['std'],
    aggregated_stats_static_side['test_macro_f1']['std']
]

bars2 = axes[1].bar(model_names, f1_means, yerr=f1_stds, capsize=5, color=colors, alpha=0.8)
axes[1].set_ylabel('Test Macro F1', fontsize=12)
axes[1].set_title('All LSTM Models - Macro F1 Comparison', fontsize=14)
axes[1].set_ylim(0, 1)
axes[1].axhline(y=max(f1_means), color='red', linestyle='--', alpha=0.5, label='Best')

# Add value labels
for bar, mean, std in zip(bars2, f1_means, f1_stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Experiment 7: LSTM Models Comparison (30 runs each)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## Temporal vs Static Comparison

In [ ]:
# Grouped bar chart: Temporal vs Static per view
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(2)  # Front, Side
width = 0.35

# Accuracy subplot
temporal_acc = [aggregated_stats_temporal_front['test_accuracy']['mean'],
                aggregated_stats_temporal_side['test_accuracy']['mean']]
static_acc = [aggregated_stats_static_front['test_accuracy']['mean'],
              aggregated_stats_static_side['test_accuracy']['mean']]

temporal_acc_std = [aggregated_stats_temporal_front['test_accuracy']['std'],
                    aggregated_stats_temporal_side['test_accuracy']['std']]
static_acc_std = [aggregated_stats_static_front['test_accuracy']['std'],
                  aggregated_stats_static_side['test_accuracy']['std']]

axes[0].bar(x - width/2, temporal_acc, width, yerr=temporal_acc_std, label='Temporal', capsize=5, color='steelblue')
axes[0].bar(x + width/2, static_acc, width, yerr=static_acc_std, label='Static', capsize=5, color='darkorange')
axes[0].set_xlabel('View')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Temporal vs Static - Accuracy')
axes[0].set_xticks(x)
axes[0].set_xticklabels(['Front', 'Side'])
axes[0].legend()
axes[0].set_ylim(0, 1)

# Macro F1 subplot
temporal_f1 = [aggregated_stats_temporal_front['test_macro_f1']['mean'],
               aggregated_stats_temporal_side['test_macro_f1']['mean']]
static_f1 = [aggregated_stats_static_front['test_macro_f1']['mean'],
             aggregated_stats_static_side['test_macro_f1']['mean']]

temporal_f1_std = [aggregated_stats_temporal_front['test_macro_f1']['std'],
                   aggregated_stats_temporal_side['test_macro_f1']['std']]
static_f1_std = [aggregated_stats_static_front['test_macro_f1']['std'],
                 aggregated_stats_static_side['test_macro_f1']['std']]

axes[1].bar(x - width/2, temporal_f1, width, yerr=temporal_f1_std, label='Temporal', capsize=5, color='steelblue')
axes[1].bar(x + width/2, static_f1, width, yerr=static_f1_std, label='Static', capsize=5, color='darkorange')
axes[1].set_xlabel('View')
axes[1].set_ylabel('Test Macro F1')
axes[1].set_title('Temporal vs Static - Macro F1')
axes[1].set_xticks(x)
axes[1].set_xticklabels(['Front', 'Side'])
axes[1].legend()
axes[1].set_ylim(0, 1)

plt.suptitle('Temporal vs Static LSTM Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## Summary Statistics

In [ ]:
# Final summary
print("=" * 100)
print("EXPERIMENT 7 - LSTM MODELS FINAL SUMMARY")
print("=" * 100)
print("\n📊 Model Performance Summary (30 runs each):\n")

for model_name, stats in all_models_stats.items():
    print(f"🔹 {model_name}:")
    print(f"   Accuracy: {stats['test_accuracy']['mean']:.4f} ± {stats['test_accuracy']['std']:.4f} "
          f"(range: {stats['test_accuracy']['min']:.4f} - {stats['test_accuracy']['max']:.4f})")
    print(f"   Macro F1: {stats['test_macro_f1']['mean']:.4f} ± {stats['test_macro_f1']['std']:.4f} "
          f"(range: {stats['test_macro_f1']['min']:.4f} - {stats['test_macro_f1']['max']:.4f})")
    print()

print("=" * 100)
print(f"\n🏆 Best Model by Mean Macro F1: {best_model_name}")
print(f"   Mean F1: {all_models_stats[best_model_name]['test_macro_f1']['mean']:.4f}")
print("=" * 100)

## Key Findings

Run this cell after training to see the conclusions.

In [ ]:
# Compute differences
temporal_avg_f1 = (aggregated_stats_temporal_front['test_macro_f1']['mean'] + 
                   aggregated_stats_temporal_side['test_macro_f1']['mean']) / 2
static_avg_f1 = (aggregated_stats_static_front['test_macro_f1']['mean'] + 
                 aggregated_stats_static_side['test_macro_f1']['mean']) / 2

front_avg_f1 = (aggregated_stats_temporal_front['test_macro_f1']['mean'] + 
                aggregated_stats_static_front['test_macro_f1']['mean']) / 2
side_avg_f1 = (aggregated_stats_temporal_side['test_macro_f1']['mean'] + 
               aggregated_stats_static_side['test_macro_f1']['mean']) / 2

print("📈 Key Findings:")
print()
print(f"1. Temporal vs Static (avg Macro F1):")
print(f"   Temporal: {temporal_avg_f1:.4f}")
print(f"   Static:   {static_avg_f1:.4f}")
print(f"   Difference: {abs(temporal_avg_f1 - static_avg_f1):.4f} in favor of {'Temporal' if temporal_avg_f1 > static_avg_f1 else 'Static'}")
print()
print(f"2. Front vs Side (avg Macro F1):")
print(f"   Front: {front_avg_f1:.4f}")
print(f"   Side:  {side_avg_f1:.4f}")
print(f"   Difference: {abs(front_avg_f1 - side_avg_f1):.4f} in favor of {'Front' if front_avg_f1 > side_avg_f1 else 'Side'}")